# **CIFAR10_CNN**
- ELEC 576 HW 2
- Robert Heeter
- 25 October 2023

## **Structure**:
- Purpose: Implement a PyTorch image classsification CNN neural network (LeNet5) on the cifar10 dataset

1) Set PyTorch metada
    - Seed
    - TensorFlow output (logging)
    - Whether to transfer to gpu (cuda)
2) Import data
    - Download data
    - Create data loaders with batchsize, transforms, scaling
3) Define model architecture, loss, and optimizer
4) Define test and training loops
    - Train:
        - Get next batch
        - Forward pass through model-
        - Calculate loss
        - Backward pass from loss (calculates the gradient for each parameter)
        - Optimizer: performs weight updates
        - Calculate accuracy, other stats
    - Test:
        - Calculate loss, accuracy, other stats
5) Perform training over multiple epochs
    - Each epoch:
        - Call train loop
        - Call test loop

## **Acknowledgements**:
- Worked with Arielle Sanford


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable
from torch.utils.data import DataLoader
import numpy as np

from torch.utils.tensorboard import SummaryWriter
from datetime import datetime
import os

import matplotlib.pyplot as plt

%load_ext tensorboard


In [ ]:
# 1. Set PyTorch metadata

batch_size = 128
epochs = 10
lr = 0.0001
try_cuda = True
seed = 1000

num_classes = 10

logging_interval = 10 # how many batches to wait before logging
grayscale = True

# setting up the logging
log_dir = os.path.join(os.getcwd(),'log/CIFAR10', datetime.now().strftime('%b%d_%H-%M-%S'))
writer = SummaryWriter(log_dir=log_dir)

# deciding whether to send to the cpu or not if available
if torch.cuda.is_available() and try_cuda:
    cuda = True
    torch.cuda.manual_seed(seed)
else:
    cuda = False
    torch.manual_seed(seed)
    

In [ ]:
# 2. Import data

transform = transforms.Compose([transforms.Grayscale(num_output_channels=1), transforms.ToTensor()]) 

train_dataset = datasets.CIFAR10('data', train=True, transform=transform, download=True)
test_dataset = datasets.CIFAR10('data', train=False, transform=transform, download=True)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)

def check_data_loader_dim(loader):
    # checking the dataset
    for images, labels in loader:
        print('Image batch dimensions: ', images.shape)
        print('Image label dimensions: ', labels.shape)
        break

check_data_loader_dim(train_loader)
check_data_loader_dim(test_loader)


In [ ]:
# 3. Defining model architecture, loss, and optimizer

# define configuration
in_channels = 1 # grayscale images have one channel
cuda = False # not using GPU
verbose = True

layer_1_n_filters = 32
layer_2_n_filters = 64
fc_1_n_nodes = 1024
padding = 2
pooling_size = 2
kernel_size = 5

# calculating the side length of the final activation maps
# convolutional layer output width = [(input_width - kernel_size + 2*padding)/stride] + 1
conv1_OW = ((layer_1_n_filters - kernel_size + 2*padding)/1) + 1

# max-pooling layer output width = [(input_width - pooling_size)/stride] + 1
maxpool1_OW = ((conv1_OW - pooling_size)/2) + 1

# convolutional layer output width = [(input_width - kernel_size + 2*padding)/stride] + 1
conv2_OW = ((maxpool1_OW - kernel_size + 2*padding)/1) + 1

# max-pooling layer output width = [(input_width - pooling_size)/stride] + 1
maxpool2_OW = ((conv2_OW - 2)/2) + 1

final_length = int(maxpool2_OW)

if verbose:
    print(f"final_length = {final_length}")

# define architecture
class LeNet5(nn.Module):
    def __init__(self, num_classes, grayscale=False):
        super(LeNet5, self).__init__()

        self.grayscale = grayscale
        self.num_classes = num_classes
        
        if self.grayscale:
            in_channels = 1
        else:
            in_channels = 3
  
        self.features = nn.Sequential(
            nn.Conv2d(in_channels, layer_1_n_filters, kernel_size=5, padding=2), # 32 filters with 5x5 kernel 
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(layer_1_n_filters, layer_2_n_filters, kernel_size=5, padding=2), # 64 filters with 5x5 kernel
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
    
        self.classifier = nn.Sequential(
            nn.Linear(final_length*final_length*layer_2_n_filters*in_channels, fc_1_n_nodes),
            nn.Tanh(),
            nn.Linear(fc_1_n_nodes, num_classes),
        )

    def forward(self, x):
        x = self.features(x) # send input through convolutional layers
        x = x.view(x.size(0), -1) # reshaping
        x = self.classifier(x) # send input through MLP layers

        logits = x # unnormalized
        probas = F.softmax(logits, dim=1) # normalized

        return logits, probas
    
model = LeNet5(num_classes=num_classes, grayscale=True)
print(model)

optimizer = optim.Adam(model.parameters(), lr=lr)
# optimizer = optim.SGD(model.parameters(), lr=lr)

print(optimizer)


In [ ]:
# 4. Define test and training loops

def train(epoch):
    model.train()

    criterion = nn.CrossEntropyLoss()
    total_loss = 0.0
    correct = 0

    for batch_idx, (data, target) in enumerate(train_loader):
        if cuda:
            data, target = data.cuda(), target.cuda()
        optimizer.zero_grad()
        logits, probas = model(data) # forward pass
        loss = criterion(logits, target)
        loss.backward() # backward pass
        optimizer.step()
        total_loss += loss.item()

        # calculate training accuracy
        _, predicted = probas.max(1)
        correct += predicted.eq(target).sum().item()
        if batch_idx % logging_interval == 0:
            print(f'Train Epoch: {epoch} [{batch_idx * len(data)}/{len(train_loader.dataset)} ({100. * batch_idx / len(train_loader):.0f}%)]\tLoss: {loss.item()}')

    # calculate and log training metrics
    train_loss = total_loss / len(train_loader.dataset)
    train_accuracy = 100. * correct / len(train_loader.dataset)
    print(f"Epoch {epoch}: Training Loss: {train_loss:.4f}, Accuracy: {train_accuracy:.2f}%")
        
    # add to TensorBoard
    writer.add_scalar('Loss/Train', train_loss, epoch)
    writer.add_scalar('Accuracy/Train', train_accuracy, epoch)

def test(epoch):
    model.eval()

    criterion = nn.CrossEntropyLoss()
    test_loss = 0
    correct = 0
    
    with torch.no_grad():
        for data, target in test_loader:
            if cuda:
                data, target = data.cuda(), target.cuda()
            logits, probas = model(data)
            test_loss += criterion(logits, target).item()
            _, predicted = probas.max(1)
            correct += predicted.eq(target).sum().item()

    # calculate and log testing metrics
    test_loss /= len(test_loader.dataset)
    test_accuracy = 100. * correct / len(test_loader.dataset)
    print(f"Test Loss: {test_loss:.4f}, Accuracy: {test_accuracy:.2f}%")
    
    # add to TensorBoard
    writer.add_scalar('Accuracy/Test', test_accuracy, epoch)


In [ ]:
# 5. Perform training over multiple epochs

# start training
for epoch in range(1, epochs + 1):
    train(epoch)
    test(epoch)

writer.close()


In [ ]:
%tensorboard --logdir log/CIFAR10 --port=8008


In [ ]:
# display weight filters

# access the first convolutional layer
first_conv_layer = model.features[0]

# get the weight data from the layer
filters = first_conv_layer.weight.data

# normalize the weights to visualize them
filters = filters - filters.min()
filters = filters / filters.max()

# plot and visualize the filters
fig, axes = plt.subplots(4, 8, figsize=(12, 6))
for i, ax in enumerate(axes.ravel()):
    ax.imshow(filters[i].cpu().numpy()[0], cmap='gray')
    ax.axis('off')

plt.show()


In [ ]:
# display activation statistics

model.eval()  # Set the model to evaluation mode

conv_layer = model.features[0]  # Assuming you want statistics for the first convolutional layer
activation_stats = []  # To store statistics

with torch.no_grad():
    for data, _ in test_loader:
        if cuda:
            data = data.cuda()

        # pass the test data through the first convolutional layer
        activations = conv_layer(data)

        # calculate statistics (mean, standard deviation)
        mean = activations.mean().item()
        std = activations.std().item()

        activation_stats.append((mean, std))

activation_stats = torch.tensor(activation_stats)

plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.plot(activation_stats[:, 0].numpy())
plt.title("Mean Activation")
plt.xlabel("Test Images")
plt.subplot(1, 2, 2)
plt.plot(activation_stats[:, 1].numpy())
plt.title("Stdev Activation")
plt.xlabel("Test Images")
plt.show()
